<a href="https://colab.research.google.com/github/chunsubin/my-first-blog/blob/master/2021_mini_drone_A_league.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **set_threshold_with_hsv.m**

---

In [ ]:
% Tello drone object
drone = ryze();

% Camera object
cam = camera(drone);

% init treshold
th_down = 0.5;
th_up = 0.55;


while 1
    % Capture an image from the drone's camera
    frame = snapshot(cam);
    subplot(2,1,1), subimage(frame);
    pause(1);
    
    
    % Get the hue data of the image
    hsv = rgb2hsv(frame);
    h = hsv(:,:,1);
    
    
    % imshow current binary image
    if (th_up - th_down) < 0
        binary_res = (th_down<h)+(h<th_up);
    else
        binary_res = (th_down<h)&(h<th_up);
    end
    subplot(2,1,2), subimage(binary_res);                                           
    disp("th_down: " + th_down + "   th_up: " + th_up);
    
    
    % keyboard input & adjust the threshold value
    x = input("(quit: q, up: e, down: d) \ninput: ", 's'); disp(newline);
    if x == 'q'
        disp("* final th_down: " + th_down + "   fianl th_up: " + th_up);
        break
    elseif x == 'e'
        th_down = th_down + 0.025;
        th_up = th_up + 0.025;
    elseif x == 'd'
        th_down = th_down - 0.025;
        th_up = th_up - 0.025;
    end
    
    if th_down > 1
        th_down = th_down - 1;
    elseif th_down < 0
        th_down = th_down + 1;
    end  
    
    if th_up > 1
        th_up = th_up - 1;
    elseif th_up < 0
        th_up = th_up + 1;
    end
end

# **control_with_img_process.m**

---

In [ ]:
try
    drone = ryze();
    cam = camera(drone);

    % (1) takeoff
    takeoff(drone);
    pause(3);

    % (2) moveright & search red color
    while 1
        moveright(drone, 'Distance', 0.2);
        pause(3);

        frame = snapshot(cam);
        pause(2);

        hsv = rgb2hsv(frame);
        h = hsv(:,:,1);
        detect_red = (h>1)+(h<0.05);

        if sum(detect_red, 'all') >= 17000
            % red color detected
            break
        end
    end

    % (3) moveup & search green color
    while 1
        moveup(drone, 'Distance', 0.2);
        pause(3);

        frame = snapshot(cam);
        pause(2);

        hsv = rgb2hsv(frame);
        h = hsv(:,:,1);
        detect_green = (0.275<h)&(h<0.325);

        if sum(detect_green, 'all') >= 14000
            % green color detected
            break
        end
    end

    % (4) moveright & search blue color
    while 1
        moveright(drone, 'Distance', 0.2);
        pause(3);

        frame = snapshot(cam);
        pause(2);

        hsv = rgb2hsv(frame);
        h = hsv(:,:,1);
        detect_blue = (0.575<h)&(h<0.625);

        if sum(detect_blue, 'all') >= 15000
            % blue color detected
            break
        end
    end

    % (5) land
    land(drone);
    
    
catch error
    disp(error);
    clear;
    
end